In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df = pd.read_csv('energydata_complete.csv')

In [21]:
from sklearn import metrics
x = df.iloc[:, 5].values
y = df.iloc[:, 13].values

x = x.reshape(-1,1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_train = x_train.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
lm = LinearRegression()
lm.fit(x_train, y_train)
predicted_values = lm.predict(x_test)

from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.71

In [15]:

df.drop(['lights','date'], axis=1,inplace=True)


KeyError: "['lights' 'date'] not found in axis"

In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


normalized_features = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)



X = normalized_features.drop(['Appliances'], axis=1)


y = normalized_features['Appliances']
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [6]:
# Instantiate the LinearRegression Model
linReg = LinearRegression()

# fit the model with training set
linReg.fit(X_train, y_train)

# Make predictions on testing set
prediction = linReg.predict(X_test)

# Computing the Mean_absolute_error
MAE = round(mean_absolute_error(y_test, prediction), 2)

MAE

0.05

In [7]:
# Computing the Residual Sum of Squares in 2 D.p
RSS = round(np.sum(np.square(y_test - prediction)), 2)

RSS

45.35

In [8]:
#Root Mean Square Error
RMSE = round(np.sqrt(metrics.mean_squared_error(y_test,prediction)),3)

RMSE

0.088

In [10]:
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

# Execute the get_weight function and store in dataframe
linear_weights_df = get_weights_df(linReg, X_train, 'Linear_weight')

linear_weights_df

,Features,Linear_weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [18]:

ridge = Ridge()

ridge.fit(X_train, y_train)


pred1 = ridge.predict(X_test)

ridge_rmse = round(np.sqrt(metrics.mean_squared_error(y_test,pred1)),3)

print(ridge_rmse)


ridge_reg = Ridge(alpha=0.4)

# fit on training set
ridge_reg.fit(X_train, y_train)


pred2 = ridge_reg.predict(X_test)

ridge_rmse_new = round(np.sqrt(metrics.mean_squared_error(y_test,pred2)),3)

print(ridge_rmse_new)

0.088
0.088


In [12]:
# Instatiate a lasso regressor with alpha value of 0.001
lasso_reg = Lasso(alpha=0.001)

# Fit the lasso model with training set
lasso_reg.fit(X_train, y_train)

# Make predictions on test set
lasso_preds = lasso_reg.predict(X_test)

# defining a function to get weights
def get_weights_df(model, feat, col_name):
  #this function returns the weight of every feature
  weights = pd.Series(model.coef_, feat.columns).sort_values()
  weights_df = pd.DataFrame(weights).reset_index()
  weights_df.columns = ['Features', col_name]
  weights_df[col_name].round(3)
  return weights_df

lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')


non_zero_weights = lasso_weights_df[lasso_weights_df['Lasso_weight'] != 0]


print(non_zero_weights)

     Features  Lasso_weight
0      RH_out     -0.049557
1        RH_8     -0.000110
24  Windspeed      0.002912
25       RH_1      0.017880


In [13]:

RMSE_new = round(np.sqrt(metrics.mean_squared_error(y_test,lasso_preds)),3)
RMSE_new

0.094